In [1]:
from pathlib import Path
import os
# !pip install iesopt > /dev/null

In [3]:
import iesopt

INFO:iesopt:Integrated Energy System Optimization (IESopt)
INFO:iesopt:    © 2021 - now:  AIT Austrian Institute of Technology GmbH
INFO:iesopt:    Documentation: https://ait-energy.github.io/iesopt
INFO:iesopt:    Target for juliapkg: 'C:\Users\ReiszP\Desktop\iesopt-trainings\twinvector-summerschool-2025\.iesopt'
INFO:iesopt:Checking Julia environment
INFO:iesopt:Disabling SSL verification to prevent problems; this may be unsafe


[juliapkg] Found dependencies: c:\users\reiszp\desktop\iesopt-trainings\twinvector-summerschool-2025\.iesopt\juliapkg.json
[juliapkg] Found dependencies: c:\users\reiszp\desktop\iesopt-trainings\twinvector-summerschool-2025\.venv\lib\site-packages\juliacall\juliapkg.json
[juliapkg] Found dependencies: c:\users\reiszp\desktop\iesopt-trainings\twinvector-summerschool-2025\.venv\lib\site-packages\juliapkg\juliapkg.json
[juliapkg] Locating Julia =1.11.5
[juliapkg] Using Julia 1.11.5 at C:\Users\ReiszP\.julia\juliaup\julia-1.11.5+0.x64.w64.mingw32\bin\julia.exe
[juliapkg] Using Julia project at c:\Users\ReiszP\Desktop\iesopt-trainings\twinvector-summerschool-2025\.venv\julia_env
[juliapkg] Writing Project.toml:
             [deps]
             JuMP = "4076af6c-e467-56ae-b986-b466b2749572"
             IESopt = "ed3f0a38-8ad9-4cf8-877e-929e8d190fe9"
             HiGHS = "87dc4568-4c63-4d18-b0c0-bb2238e4078b"
             PythonCall = "6099a3de-0909-46bc-b1f4-468b9a2dfc0d"
             OpenSS

INFO:iesopt:Julia environment ready, loading Julia
INFO:iesopt:    Using default sysimage
INFO:iesopt:    Executable: C:\Users\ReiszP\.julia\juliaup\julia-1.11.5+0.x64.w64.mingw32\bin\julia.exe
INFO:iesopt:    Project: c:\Users\ReiszP\Desktop\iesopt-trainings\twinvector-summerschool-2025\.venv\julia_env
INFO:iesopt:Julia setup complete
INFO:iesopt:Importing Julia modules:
INFO:iesopt:    IESopt
INFO:iesopt:    JuMP
INFO:iesopt:Loaded versions: py=2.8.0, jl=2.8.0


# Running the model

After setting up everything in the YAMLs (in the `opt/` folder), we call ` iesopt.run(...)`, that runs the optimization problem.
The forst argument is the main YAML - config file. The argument `parameters` allows us to modify the parameters that we have defined in the YAMLs previously.

For example `price_electricity_buy = "dayahead_price@data", price_electricity_sell = 10.0` 

would model buying at the day-ahead price and selling (feed-in) at a fixed price of 10.0 EUR/MWh.

In [9]:
model = iesopt.run(
    "opt/config_summerschool2025.iesopt.yaml",
    parameters=dict(
        # price_electricity_buy = "dayahead_price@data", price_electricity_sell = 10.0 
    ),
)

## Convert and extract the results

After we have the solution of the problem (i.e. set-point powers of the single components at the given timesteps) we extract and save the for the subsequent post-processing in csv format into the `opt/out` folder.

On extracting the results from the solved IESopt model, you can also consult the [documentation](https://ait-energy.github.io/iesopt/notebooks/custom_results_1.html#accessing-model-results-objectives).

In [10]:
def extract_results_to_csv(model, output_subdir =""):
    df_results = model.results.to_pandas()

    # list components
    print( "## MODEL COMPONENTs: ##\n", df_results["component"].unique() )
    output_dir = Path("opt/out") / output_subdir
    output_dir.mkdir(parents=True, exist_ok=True)
    # electricity buy / sell
    elec_buy = df_results.query("component == 'buy_electricity' and field == 'value'")
    elec_sold = df_results.query("component == 'sell_electricity' and field == 'value'")
    combo = elec_buy.merge(elec_sold, how="inner", on="snapshot", suffixes=("_buy", "_sell"))
    combo.to_csv(os.path.join(output_dir, "elec_buy_sell.csv"))

    # pv
    pv = df_results.query("component == 'pv' and field == 'value'")
    # load
    load = df_results.query("component == 'load' and field == 'value'") 
    combo=pv.merge(load, how="inner", on="snapshot", suffixes=("_pv", "_load"))
    combo.to_csv(os.path.join(output_dir, "pv_load.csv"))

    # battery
    batt_charge = df_results.query(
        "component.str.endswith('.charging') and fieldtype == 'exp' and field =='in_electricity'",
        engine="python"
    )
    batt_discharge = df_results.query(
        "component.str.endswith('.discharging') and fieldtype == 'exp' and field =='in_electricity'",
        engine="python"
    )
    combo =batt_charge.merge(batt_discharge, how="inner", on="snapshot", suffixes=("_charge", "_discharge"))
    combo.to_csv(os.path.join(output_dir, "batt_charge_discharge.csv"))
extract_results_to_csv(model)


## MODEL COMPONENTs: ##
 ['battery.discharging' 'battery.storage' 'pv' 'battery' 'buy_electricity'
 'load' 'sell_electricity' 'battery.charging' 'grid_electricity']


# Vizualization

The library `iesopttools` allows us to vizualize the results we obtained,

In [11]:
from iesopttools import RDB, Figure, Trace

def visualize(model):
    rdb = RDB()
    entry = rdb.add_entry(model)

    fig = Figure(style="seaborn", barmode="relative", labels=dict(title="electricity: supply / demand", x="time", y="MW"))

    for asset in entry.query("carrier", "node = 'grid_electricity' AND direction = 'out'"):
        fig.add(Trace("bar", entry.select(asset)))

    for asset in entry.query("carrier", "node = 'grid_electricity' AND direction = 'in'"):
        fig.add(Trace("bar", entry.select(asset), sign=-1.0))
        
        
    return fig

In [12]:
fig = visualize(model)
fig.render(xslice=(0,8700))
fig.show()
fig._fig.write_html("opt/out/plot_summerschool.html")


# Looping over configurations

#### The basic setup
```csv
name,type,carrier,p,e
load,Demand,electricity,5.0 * demand@data,
pv,RES,,5.0 * generation_solar@data,
```

```yaml
parameters:
  price_electricity_buy: 1e4
  price_electricity_sell: 1e4
  p_bess: 2.5
  h_bess: 10.0

  # etc ....
  battery:
    type: BESS
    p: <p_bess>
    e: <h_bess> *<p_bess>
```

This means we have in the basic a 
- 5MW PV.
- 2.5 MW - 4h BESS

In [13]:
battery_powers = [2.5, 5.0, 7.5]
battery_hours = [2.0, 4.0]

for p in battery_powers:
    for h in battery_hours:
        model = iesopt.run(
            "opt/config_summerschool2025.iesopt.yaml",
            parameters=dict(
                p_bess = p, h_bess = h, 
            ),
        )
        extract_results_to_csv(model, f"p_{p}_h_{h}")
        fig = visualize(model)
        fig.render(xslice=(0,2800))
        fig._fig.write_html(f"opt/out/p_{p}_h_{h}/plot_summerschool.html")

## MODEL COMPONENTs: ##
 ['battery.discharging' 'battery.storage' 'pv' 'battery' 'buy_electricity'
 'load' 'sell_electricity' 'battery.charging' 'grid_electricity']
## MODEL COMPONENTs: ##
 ['battery.discharging' 'battery.storage' 'pv' 'battery' 'buy_electricity'
 'load' 'sell_electricity' 'battery.charging' 'grid_electricity']
## MODEL COMPONENTs: ##
 ['battery.discharging' 'battery.storage' 'pv' 'battery' 'buy_electricity'
 'load' 'sell_electricity' 'battery.charging' 'grid_electricity']
## MODEL COMPONENTs: ##
 ['battery.discharging' 'battery.storage' 'pv' 'battery' 'buy_electricity'
 'load' 'sell_electricity' 'battery.charging' 'grid_electricity']
## MODEL COMPONENTs: ##
 ['battery.discharging' 'battery.storage' 'pv' 'battery' 'buy_electricity'
 'load' 'sell_electricity' 'battery.charging' 'grid_electricity']
## MODEL COMPONENTs: ##
 ['battery.discharging' 'battery.storage' 'pv' 'battery' 'buy_electricity'
 'load' 'sell_electricity' 'battery.charging' 'grid_electricity']
